<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Red%20neuronal%20que%20lee%20los%20features%20y%20no%20el%20text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from google.colab import files

!pip install -q tensorflow-hub
import tensorflow_hub as hub

url1 = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/train-features.csv'
url2 = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test-features.csv'

train_df = pd.read_csv(url1)
test_df = pd.read_csv(url2)

train_df.head(5)

,Unnamed: 0,id,keyword,location,text,target,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0,0,0,0,0,False,0,0,False,7,False,False
1,1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,1,0,0,0,0,False,0,0,False,3,True,False
2,2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,0,0,0,0,0,False,0,0,False,13,False,False
3,3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1,0,0,0,0,False,0,0,False,2,False,False
4,4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1,0,0,0,0,False,0,0,False,10,False,False


In [ ]:
train_df = train_df.drop(['Unnamed: 0', 'id', 'keyword', 'location', 'text'],axis = 1)
train_target_df = train_df.pop('target')
test_df = test_df.drop(['Unnamed: 0', 'keyword', 'location', 'text'],axis = 1)
test_id_df = test_df.pop('id')

train_df.shape

(7613, 14)

In [ ]:
train_df.head()

In [ ]:
train_df['has_mentions'] = train_df['has_mentions'].astype(int)
train_df['has_link'] = train_df['has_link'].astype(int)
train_df['contains_countries'] = train_df['contains_countries'].astype(int)
train_df['contains_cities'] = train_df['contains_cities'].astype(int)

test_df['has_mentions'] = test_df['has_mentions'].astype(int)
test_df['has_link'] = test_df['has_link'].astype(int)
test_df['contains_countries'] = test_df['contains_countries'].astype(int)
test_df['contains_cities'] = test_df['contains_cities'].astype(int)

train_df.head()

,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,69,13,0,0,0,0,0,0,0,0,0,7,0,0
1,38,7,1,0,0,0,0,0,0,0,0,3,1,0
2,133,22,0,0,0,0,0,0,0,0,0,13,0,0
3,65,8,1,0,0,0,0,0,0,0,0,2,0,0
4,88,16,1,0,0,0,0,0,0,0,0,10,0,0


In [ ]:
# ARMAMOS EL MODELO
model = tf.keras.Sequential()
model.add(tf.keras.layers.LayerNormalization(input_shape = tuple([train_df.shape[1]])))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.LayerNormalization())
model.add(tf.keras.layers.Dense(14, activation = 'relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_normalization_2 (Layer (None, 14)                28        
_________________________________________________________________
dense_3 (Dense)              (None, 64)                960       
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
layer_normalization_3 (Layer (None, 64)                128       
_________________________________________________________________
dense_4 (Dense)              (None, 14)                910       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 15        
Total params: 2,041
Trainable params: 2,041
Non-trainable params: 0
____________________________________________________

In [ ]:
# COMPILAMOS Y ENTRENAMOS
train_features_list = np.asarray(train_df)
train_target_list = np.asarray(train_target_df)

model.compile(optimizer = 'adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics = ['accuracy'])

history = model.fit(train_features_list, train_target_list, batch_size=1024,
                    epochs = 512,
                    verbose = 1)

Epoch 1/512
8/8 [==============================] - 0s 8ms/step - loss: 0.7707 - accuracy: 0.5162
Epoch 2/512
8/8 [==============================] - 0s 6ms/step - loss: 0.7238 - accuracy: 0.5686
Epoch 3/512
8/8 [==============================] - 0s 6ms/step - loss: 0.7090 - accuracy: 0.5657
Epoch 4/512
8/8 [==============================] - 0s 7ms/step - loss: 0.7006 - accuracy: 0.5618
Epoch 5/512
8/8 [==============================] - 0s 6ms/step - loss: 0.7024 - accuracy: 0.5640
Epoch 6/512
8/8 [==============================] - 0s 5ms/step - loss: 0.6903 - accuracy: 0.5718
Epoch 7/512
8/8 [==============================] - 0s 6ms/step - loss: 0.6936 - accuracy: 0.5710
Epoch 8/512
8/8 [==============================] - 0s 7ms/step - loss: 0.6896 - accuracy: 0.5684
Epoch 9/512
8/8 [==============================] - 0s 6ms/step - loss: 0.6872 - accuracy: 0.5710
Epoch 10/512
8/8 [==============================] - 0s 6ms/step - loss: 0.6869 - accuracy: 0.5699
Epoch 11/512
8/8 [===========

In [ ]:
# EVALUAMOS CON EL SET DE VALIDACION
loss, precision = model.evaluate(train_features_list, train_target_list)
print('Precisión: %.2f' % (precision*100))

238/238 [==============================] - 0s 1ms/step - loss: 0.5892 - accuracy: 0.6547
Precisión: 65.47


In [ ]:
# ARMAMOS LA PREDICCIÓN
prediction = model.predict_classes(test_df, batch_size=512)
prediction

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[0],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]], dtype=int32)

In [ ]:
test_df

,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,34,6,0,0,0,0,0,0,0,0,0,4,0,0
1,64,9,0,0,0,0,0,0,0,0,0,7,0,0
2,96,19,1,0,0,0,0,0,0,0,0,10,0,0
3,40,4,1,0,0,0,0,0,0,0,0,3,0,1
4,45,8,0,0,0,0,0,0,0,0,0,6,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3258,55,8,0,0,0,0,0,0,0,0,0,0,0,0
3259,139,23,0,0,0,0,0,0,0,0,0,8,0,0
3260,55,6,0,0,0,0,0,0,0,0,1,4,0,1
3261,65,7,0,0,0,0,0,0,0,0,1,4,0,0


In [ ]:
# EXPORTAMOS EL CSV EN EL FORMATO QUE PIDE KAGGLE
test_df['target'] = prediction
export = pd.DataFrame()
export['id'] = test_id_df['id']
export['target'] = test_df['target']
export = export.set_index('id')
export.to_csv('submit.csv')
files.download('submit.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>